In [48]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
tqdm.pandas()

# Подготовка данных

In [49]:
df = pd.read_csv('rusentitweet_full.csv').drop('id', axis=1)
df.label = df.label.map({'negative': 0, 'positive': 1})
df.dropna(inplace=True)
df.label = df.label.astype(int)

## Очистка
1. теги и ссылки заменяем на специальные слова
2. знаки !, ? заменяем на специальные слова
2. переносы строк заменяем на пробельный символ

#### Улучшения:
1. Смайлики заменяются на слова
2. Цифрцы убираются полностью (тк не содержат в себе окраски)

In [50]:
import re
import demoji

def clean_text(text: str) -> str:
    text = text.replace('\r\n', ' ')
    text = text.replace('ё', 'е')
    text = re.sub('@[^\s]+', 'tag', text) # заменяем теги на специальное слово
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'url', text) # заменяем урлы на специальное слово
    text = demoji.replace_with_desc(text, sep=' ') # заменяем емоджи на слова

    text = text.replace('!', ' exclamation ')
    text = text.replace('?', ' question ')
    text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text)

    text = text.lower()
    return text.strip()

df["clean_text"] = df["text"].apply(clean_text)
df

,Unnamed: 0,text,label,clean_text
1,1,велл они всё равно что мусор так что ничего с...,0,велл они все равно что мусор так что ничего ст...
2,2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",0,трезвая жизнь какая то такая стремная с артем ...
8,8,@BTS_twt ты такой красивый 😭😭😭🥺💓,1,tag ты такой красивый loudly crying face loudl...
9,9,"@Ladyzchensk Цыган , хуле ...",0,tag цыган хуле
11,11,@okdaa @wifeyoonminn @Y_Yoon_ Но ты очень крас...,1,tag tag tag но ты очень красиво и чувственно п...
...,...,...,...,...
13382,13382,@_sasukedaisuki_ я не знаю что сказать просто\...,1,tag я не знаю что сказать просто two hearts tw...
13384,13384,ладно всё делаю это ваше сраное дз,0,ладно все делаю это ваше сраное дз
13385,13385,66-летняя женщина пострадала на пожаре на улиц...,0,летняя женщина пострадала на пожаре на улице к...
13387,13387,все пора спать пиздец словила шизу,0,все пора спать пиздец словила шизу


## Токенизация

In [51]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased", do_lower_case=True)

def tokenize(text: str) -> pd.Series:
    res = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=32,  # Максимальная длина входной последовательности - позволяет оптимизировать память
                        # Ограничение BERT-a - 512, но если сделать меньше, то модель будет обучаться быстрее
                        # Можно заранее посчитать максимальную длину последовательности
                        # на датасете (считать нужно в токенах по attention mask)
        pad_to_max_length=True,  # Нужно ли дополнять предложение до максимальной длины
                                 # Да, нужно - в таком случае можно делить на батчи
                                 # (если векторы будут разной размерности, упадем с ошибкой)
        return_attention_mask=True,  # Attention mask - показывает, имеет ли токен смысл
                                     # токен [PAD] - 0, остальные - 1
        return_tensors="pt"  # Указываем тип тензоров, нам нуше PyTorch
    )
    return pd.Series([res["input_ids"], res["attention_mask"]])

df[["input_ids", "attention_mask"]] = df["clean_text"].progress_apply(tokenize)

  0%|          | 0/5712 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\MarkB\projects\ProjectsPycharm\ml\venv\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## Упаковка датасета

In [52]:
test_size = 0.3
batch_size = 16

# Делим выборку на трейн и тест со стратификацией - сохраняя распределение классов
train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    shuffle=True,
    stratify=df["label"].values
)

# Train and validation sets
train_set = TensorDataset(torch.cat(list(train_df["input_ids"].values), dim=0),
                          torch.cat(list(train_df["attention_mask"].values), dim=0),
                          torch.tensor(train_df["label"].values))

test_set = TensorDataset(torch.cat(list(test_df["input_ids"].values), dim=0),
                         torch.cat(list(test_df["attention_mask"].values), dim=0),
                         torch.tensor(test_df["label"].values))

# Prepare DataLoader. Позволяет итерироваться по тензорам
train_dataloader = DataLoader(
            train_set,
            batch_size=batch_size
        )

test_dataloader = DataLoader(
            test_set,
            batch_size=batch_size
        )

# Процедура дообучения

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [54]:
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels = 2, # 1 и 0
    output_attentions = False,
    output_hidden_states = False, # отключает ембединги
)

model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [55]:
epochs = 5 # ну пусть пять...

# чтобы можно было веса не высчитывать
# он обновляет веса
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-6,
    eps = 1e-08 # линейный сдвиг?
)


for _ in tqdm(range(epochs), desc='Epoch'):
    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)): # итерируемся по датасету
        # batch = batch.type(torch.LongTensor)
        batch = tuple(t.type(torch.LongTensor).to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad() # обновляем градиент

        # Forward pass
        train_output = model(b_input_ids,
                             token_type_ids = None,
                             attention_mask = b_input_mask,
                             labels = b_labels)

        # Backward pass
        train_output.loss.backward() # gросчитываем ф-ю потерь
        optimizer.step() # просчитываем веса

        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========
    # проверяем ее на тех данных, которые она не видела

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_f1 = []

    for batch in tqdm(test_dataloader, total=len(test_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # градиенты не пересчитываем

        with torch.no_grad():
          eval_output = model(b_input_ids,
                              token_type_ids = None,
                              attention_mask = b_input_mask)

        logits = eval_output.logits
        y_pred = torch.argmax(logits, dim = -1)

        y_pred = y_pred.detach().cpu().numpy()
        y_true = b_labels.to('cpu').numpy()

        # Calculate validation metrics
        val_f1_value = f1_score(y_true, y_pred, average='macro') # микро - для работы с точечными значениями, у нас же важно как оно
        # выглядит на фоне всех комментариев
        val_f1.append(val_f1_value)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation F1-score: {:.4f}'.format(sum(val_f1)/len(val_f1)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]


	 - Train loss: 0.5771
	 - Validation F1-score: 0.7914


  0%|          | 0/250 [00:00<?, ?it/s]

...

  0%|          | 0/108 [00:00<?, ?it/s]


	 - Train loss: 0.1776
	 - Validation F1-score: 0.8274


  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]


	 - Train loss: 0.1238
	 - Validation F1-score: 0.7976


# Inference модели

In [60]:
def is_positive(input_str: str) -> bool:
    if model.training:
        raise RuntimeError("Model mustn't be in training mode, use model.eval()")

    text = clean_text(input_str)
    series = tokenize(text)

    ts = TensorDataset(series[0], series[1])
    dl = DataLoader(ts, batch_size=1)

    dl_iter = iter(dl)
    btch = next(dl_iter)

    btch = tuple(t.to(device) for t in btch)
    input_ids, input_mask = btch

    eval_outputt = model(input_ids, token_type_ids=None, attention_mask=input_mask)
    logitss = eval_outputt.logits
    y_predd = torch.argmax(logitss, dim=-1)
    y_predd = y_predd.detach().cpu().numpy()

    res = bool(y_predd[0])
    print(f"{input_str} -> {'Positive' if res else 'Negative'}")
    return res

In [63]:
is_positive("вы крутые ребята")
is_positive("вы че там ахуели совсем??")
is_positive("замеля ему стекловатой")


C:\Users\MarkB\projects\ProjectsPycharm\ml\venv\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


вы крутые ребята -> Positive
вы че там ахуели совсем?? -> Negative
замеля ему стекловатой -> Negative


False